In [1]:
!pip install optuna

In [1]:
import optuna

In [3]:
!pip install xgboost

In [30]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [31]:
df = pd.read_csv('Admission_Prediction.csv')

In [32]:
df

Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  \
0             1      337.0        118.0                4.0  4.5  4.5  9.65   
1             2      324.0        107.0                4.0  4.0  4.5  8.87   
2             3        NaN        104.0                3.0  3.0  3.5  8.00   
3             4      322.0        110.0                3.0  3.5  2.5  8.67   
4             5      314.0        103.0                2.0  2.0  3.0  8.21   
..          ...        ...          ...                ...  ...  ...   ...   
495         496      332.0        108.0                5.0  4.5  4.0  9.02   
496         497      337.0        117.0                5.0  5.0  5.0  9.87   
497         498      330.0        120.0                5.0  4.5  5.0  9.56   
498         499      312.0        103.0                4.0  4.0  5.0  8.43   
499         500      327.0        113.0                4.0  4.5  4.5  9.04   

     Research  Chance of Admit  
0           1             0.92  
1           1             0.76  
2           1             0.72  
3           1             0.80  
4           0             0.65  
..        ...              ...  
495         1             0.87  
496         1             0.96  
497         1             0.93  
498         0             0.73  
499         0             0.84  

[500 rows x 9 columns]

In [33]:
df.describe()

Serial No.   GRE Score  TOEFL Score  University Rating         SOP  \
count  500.000000  485.000000   490.000000         485.000000  500.000000   
mean   250.500000  316.558763   107.187755           3.121649    3.374000   
std    144.481833   11.274704     6.112899           1.146160    0.991004   
min      1.000000  290.000000    92.000000           1.000000    1.000000   
25%    125.750000  308.000000   103.000000           2.000000    2.500000   
50%    250.500000  317.000000   107.000000           3.000000    3.500000   
75%    375.250000  325.000000   112.000000           4.000000    4.000000   
max    500.000000  340.000000   120.000000           5.000000    5.000000   

             LOR        CGPA    Research  Chance of Admit  
count  500.00000  500.000000  500.000000        500.00000  
mean     3.48400    8.576440    0.560000          0.72174  
std      0.92545    0.604813    0.496884          0.14114  
min      1.00000    6.800000    0.000000          0.34000  
25%      3.00000    8.127500    0.000000          0.63000  
50%      3.50000    8.560000    1.000000          0.72000  
75%      4.00000    9.040000    1.000000          0.82000  
max      5.00000    9.920000    1.000000          0.97000

In [34]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [35]:
x= df.drop(columns=['Serial No.' ,'Chance of Admit' ])
x

GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  Research
0    337.000000        118.0                4.0  4.5  4.5  9.65         1
1    324.000000        107.0                4.0  4.0  4.5  8.87         1
2    316.558763        104.0                3.0  3.0  3.5  8.00         1
3    322.000000        110.0                3.0  3.5  2.5  8.67         1
4    314.000000        103.0                2.0  2.0  3.0  8.21         0
..          ...          ...                ...  ...  ...   ...       ...
495  332.000000        108.0                5.0  4.5  4.0  9.02         1
496  337.000000        117.0                5.0  5.0  5.0  9.87         1
497  330.000000        120.0                5.0  4.5  5.0  9.56         1
498  312.000000        103.0                4.0  4.0  5.0  8.43         0
499  327.000000        113.0                4.0  4.5  4.5  9.04         0

[500 rows x 7 columns]

In [36]:
x.describe()

GRE Score  TOEFL Score  University Rating         SOP        LOR  \
count  500.000000   500.000000         500.000000  500.000000  500.00000   
mean   316.558763   107.187755           3.121649    3.374000    3.48400   
std     11.103952     6.051338           1.128802    0.991004    0.92545   
min    290.000000    92.000000           1.000000    1.000000    1.00000   
25%    309.000000   103.000000           2.000000    2.500000    3.00000   
50%    316.558763   107.000000           3.000000    3.500000    3.50000   
75%    324.000000   112.000000           4.000000    4.000000    4.00000   
max    340.000000   120.000000           5.000000    5.000000    5.00000   

             CGPA    Research  
count  500.000000  500.000000  
mean     8.576440    0.560000  
std      0.604813    0.496884  
min      6.800000    0.000000  
25%      8.127500    0.000000  
50%      8.560000    1.000000  
75%      9.040000    1.000000  
max      9.920000    1.000000

In [37]:
df.isnull().sum()


Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [38]:
y = df['Chance of Admit' ]
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [39]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x   = std_sca.fit_transform(x)

In [40]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [41]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        'tree_method':'gpu_hist',
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

{'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [42]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [45]:
!NVIDIA-SMI

Sun Oct 31 16:23:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.68       Driver Version: 471.68       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:0A:00.0  On |                  N/A |
| 38%   54C    P2    68W / 300W |   2901MiB / 11264MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [44]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 10)
find_param.best_trial.params

[I 2021-10-31 16:17:35,364] A new study created in memory with name: no-name-76bb0223-3b63-435e-bfd9-2e89621d426b


[16:17:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.09562
[1]	validation_0-rmse:0.08868
[2]	validation_0-rmse:0.08274
[3]	validation_0-rmse:0.08580
[4]	validation_0-rmse:0.08463
[5]	validation_0-rmse:0.08052
[6]	validation_0-rmse:0.08052
[7]	validation_0-rmse:0.08465
[8]	validation_0-rmse:0.08427
[9]	validation_0-rmse:0.08427
[10]	validation_0-rmse:0.08187
[11]	validation_0-rmse:0.08281
[12]	validation_0-rmse:0.08020
[13]	validation_0-rmse:0.08149
[14]	validation_0-rmse:0.08122
[15]	validation_0-rmse:0.08027
[16]	validation_0-rmse:0.07908
[17]	validation_0-rmse:0.07878
[18]	validation_0-rmse:0.07818
[

[I 2021-10-31 16:17:35,900] Trial 0 finished with value: 0.006186754802195331 and parameters: {'lambda': 0.5646022891371127, 'alpha': 0.35848071012137483, 'colsample_bytree': 0.9, 'subsample': 0.2, 'learning_rate': 1, 'max_depth': 11, 'random_state': 30, 'min_child_weight': 30}. Best is trial 0 with value: 0.006186754802195331.


[16:17:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.23287
[1]	validation_0-rmse:0.23077
[2]	validation_0-rmse:0.22880
[3]	validation_0-rmse:0.22686
[4]	validation_0-rmse:0.22490
[5]	validation_0-rmse:0.22290
[6]	validation_0-rmse:0.22102
[7]	validation_0-rmse:0.21918
[8]	validation_0-rmse:0.21723
[9]	validation_0-rmse:0.21538
[10]	validation_0-rmse:0.21360
[11]	validation_0-rmse:0.21184
[12]	validation_0-rmse:0.21005
[13]	validation_0-rmse:0.20821
[14]	validation_0-rmse:0.20646
[15]	validation_0-rmse:0.20479
[16]	validation_0-rmse:0.20314
[17]	validation_0-rmse:0.20145
[18]	validation_0-rmse:0.19984
[

[I 2021-10-31 16:17:36,367] Trial 1 finished with value: 0.013586775818411832 and parameters: {'lambda': 2.9931782606996893, 'alpha': 0.004737700115569739, 'colsample_bytree': 0.2, 'subsample': 1, 'learning_rate': 0.01, 'max_depth': 12, 'random_state': 20, 'min_child_weight': 138}. Best is trial 0 with value: 0.006186754802195331.


...

[16:17:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.23499
[1]	validation_0-rmse:0.23499
[2]	validation_0-rmse:0.23499
[3]	validation_0-rmse:0.23499
[4]	validation_0-rmse:0.23499
[5]	validation_0-rmse:0.23499
[6]	validation_0-rmse:0.23499
[7]	validation_0-rmse:0.23499
[8]	validation_0-rmse:0.23499
[9]	validation_0-rmse:0.23499
[10]	validation_0-rmse:0.23499
[11]	validation_0-rmse:0.23499
[12]	validation_0-rmse:0.23499
[13]	validation_0-rmse:0.23499
[14]	validation_0-rmse:0.23499
[15]	validation_0-rmse:0.23499
[16]	validation_0-rmse:0.23499
[17]	validation_0-rmse:0.23499
[18]	validation_0-rmse:0.23499
[

[I 2021-10-31 16:17:39,525] Trial 8 finished with value: 0.055220000000000005 and parameters: {'lambda': 0.014270614123445212, 'alpha': 0.04518927185267129, 'colsample_bytree': 0.1, 'subsample': 0.1, 'learning_rate': 0.01, 'max_depth': 10, 'random_state': 10, 'min_child_weight': 103}. Best is trial 0 with value: 0.006186754802195331.


[16:17:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.23492
[1]	validation_0-rmse:0.23486
[2]	validation_0-rmse:0.23479
[3]	validation_0-rmse:0.23473
[4]	validation_0-rmse:0.23467
[5]	validation_0-rmse:0.23460
[6]	validation_0-rmse:0.23454
[7]	validation_0-rmse:0.23448
[8]	validation_0-rmse:0.23442
[9]	validation_0-rmse:0.23436
[10]	validation_0-rmse:0.23429
[11]	validation_0-rmse:0.23423
[12]	validation_0-rmse:0.23416
[13]	validation_0-rmse:0.23410
[14]	validation_0-rmse:0.23404
[15]	validation_0-rmse:0.23397
[16]	validation_0-rmse:0.23392
[17]	validation_0-rmse:0.23385
[18]	validation_0-rmse:0.23379
[

[I 2021-10-31 16:17:39,986] Trial 9 finished with value: 0.052328085789056414 and parameters: {'lambda': 0.006539922941411749, 'alpha': 0.003701123608296486, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.0003, 'max_depth': 8, 'random_state': 2000, 'min_child_weight': 94}. Best is trial 0 with value: 0.006186754802195331.


{'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [ ]:
xgb.XGBRFRegressor()

In [25]:
df.describe()

Serial No.   GRE Score  TOEFL Score  University Rating         SOP  \
count  500.000000  485.000000   490.000000         485.000000  500.000000   
mean   250.500000  316.558763   107.187755           3.121649    3.374000   
std    144.481833   11.274704     6.112899           1.146160    0.991004   
min      1.000000  290.000000    92.000000           1.000000    1.000000   
25%    125.750000  308.000000   103.000000           2.000000    2.500000   
50%    250.500000  317.000000   107.000000           3.000000    3.500000   
75%    375.250000  325.000000   112.000000           4.000000    4.000000   
max    500.000000  340.000000   120.000000           5.000000    5.000000   

             LOR        CGPA    Research  Chance of Admit  
count  500.00000  500.000000  500.000000        500.00000  
mean     3.48400    8.576440    0.560000          0.72174  
std      0.92545    0.604813    0.496884          0.14114  
min      1.00000    6.800000    0.000000          0.34000  
25%      3.00000    8.127500    0.000000          0.63000  
50%      3.50000    8.560000    1.000000          0.72000  
75%      4.00000    9.040000    1.000000          0.82000  
max      5.00000    9.920000    1.000000          0.97000

In [46]:
find_param.trials_dataframe()

number     value             datetime_start          datetime_complete  \
0       0  0.006187 2021-10-31 16:17:35.366593 2021-10-31 16:17:35.900935   
1       1  0.013587 2021-10-31 16:17:35.901935 2021-10-31 16:17:36.367249   
2       2  0.012033 2021-10-31 16:17:36.368248 2021-10-31 16:17:36.876249   
3       3  0.008669 2021-10-31 16:17:36.878248 2021-10-31 16:17:37.325471   
4       4  0.016828 2021-10-31 16:17:37.326470 2021-10-31 16:17:37.785817   
5       5  0.052773 2021-10-31 16:17:37.787816 2021-10-31 16:17:38.215816   
6       6  0.053086 2021-10-31 16:17:38.217817 2021-10-31 16:17:38.659148   
7       7  0.019055 2021-10-31 16:17:38.660147 2021-10-31 16:17:39.086301   
8       8  0.055220 2021-10-31 16:17:39.088299 2021-10-31 16:17:39.524106   
9       9  0.052328 2021-10-31 16:17:39.526105 2021-10-31 16:17:39.986983   

                duration  params_alpha  params_colsample_bytree  \
0 0 days 00:00:00.534342      0.358481                      0.9   
1 0 days 00:00:00.465314      0.004738                      0.2   
2 0 days 00:00:00.508001      0.225514                      0.5   
3 0 days 00:00:00.447223      0.000700                      0.7   
4 0 days 00:00:00.459347      0.015599                      0.5   
5 0 days 00:00:00.428000      0.000113                      0.2   
6 0 days 00:00:00.441331      0.030449                      0.3   
7 0 days 00:00:00.426154      9.146701                      1.0   
8 0 days 00:00:00.435807      0.045189                      0.1   
9 0 days 00:00:00.460878      0.003701                      0.9   

   params_lambda  params_learning_rate  params_max_depth  \
0       0.564602                1.0000                11   
1       2.993178                0.0100                12   
2       9.863864                0.0100                 7   
3       0.670315                1.0000                 6   
4       0.842617                0.0080                12   
5       2.665009                0.0003                 7   
6       0.002684                0.0003                 9   
7       1.052245                1.0000                 9   
8       0.014271                0.0100                10   
9       0.006540                0.0003                 8   

   params_min_child_weight  params_random_state  params_subsample     state  
0                       30                   30               0.2  COMPLETE  
1                      138                   20               1.0  COMPLETE  
2                       41                   30               0.8  COMPLETE  
3                      175                   20               0.9  COMPLETE  
4                       55                   10               0.3  COMPLETE  
5                       56                   20               0.2  COMPLETE  
6                      172                 3454               0.4  COMPLETE  
7                      100               243123               0.2  COMPLETE  
8                      103                   10               0.1  COMPLETE  
9                       94                 2000               0.5  COMPLETE

In [47]:
optuna.visualization.plot_optimization_history(find_param)

In [48]:
optuna.visualization.plot_slice(find_param)

In [49]:
optuna.visualization.plot_contour(find_param , params = ['alpha' , 'lambda'])

In [50]:
best_param = {'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [54]:
xgb_final_model = xgb.XGBRegressor(**best_param)
xgb_final_model.fit(x,y)

XGBRegressor(alpha=0.35848071012137483, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', lambda=0.5646022891371127,
             learning_rate=1, max_delta_step=0, max_depth=11,
             min_child_weight=30, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=32, num_parallel_tree=1, predictor='auto',
             random_state=30, reg_alpha=0.358480722, reg_lambda=0.564602315,
             scale_pos_weight=1, subsample=0.2, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [55]:
xgb_final_model.score(x,y)

0.7818600803971398

In [56]:
#classification task 

In [1]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('winequality_red.csv')

In [3]:
df

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1594            6.2             0.600         0.08             2.0      0.090   
1595            5.9             0.550         0.10             2.2      0.062   
1596            6.3             0.510         0.13             2.3      0.076   
1597            5.9             0.645         0.12             2.0      0.075   
1598            6.0             0.310         0.47             3.6      0.067   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                    11.0                  34.0  0.99780  3.51       0.56   
1                    25.0                  67.0  0.99680  3.20       0.68   
2                    15.0                  54.0  0.99700  3.26       0.65   
3                    17.0                  60.0  0.99800  3.16       0.58   
4                    11.0                  34.0  0.99780  3.51       0.56   
...                   ...                   ...      ...   ...        ...   
1594                 32.0                  44.0  0.99490  3.45       0.58   
1595                 39.0                  51.0  0.99512  3.52       0.76   
1596                 29.0                  40.0  0.99574  3.42       0.75   
1597                 32.0                  44.0  0.99547  3.57       0.71   
1598                 18.0                  42.0  0.99549  3.39       0.66   

      alcohol  quality  
0         9.4        5  
1         9.8        5  
2         9.8        5  
3         9.8        6  
4         9.4        5  
...       ...      ...  
1594     10.5        5  
1595     11.2        6  
1596     11.0        6  
1597     10.2        5  
1598     11.0        6  

[1599 rows x 12 columns]

In [4]:
x = df.drop(columns='quality')
y = df.quality 

In [5]:
df.describe()

fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000              6.000000     0.990070   
25%       0.070000             7.000000             22.000000     0.995600   
50%       0.079000            14.000000             38.000000     0.996750   
75%       0.090000            21.000000             62.000000     0.997835   
max       0.611000            72.000000            289.000000     1.003690   

                pH    sulphates      alcohol      quality  
count  1599.000000  1599.000000  1599.000000  1599.000000  
mean      3.311113     0.658149    10.422983     5.636023  
std       0.154386     0.169507     1.065668     0.807569  
min       2.740000     0.330000     8.400000     3.000000  
25%       3.210000     0.550000     9.500000     5.000000  
50%       3.310000     0.620000    10.200000     6.000000  
75%       3.400000     0.730000    11.100000     6.000000  
max       4.010000     2.000000    14.900000     8.000000

In [6]:
train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )


In [7]:
np.asarray(train_x)

array([[ 8.  ,  0.39,  0.3 , ...,  3.39,  0.61,  9.  ],
       [ 6.3 ,  0.55,  0.15, ...,  3.32,  0.82, 11.6 ],
       [ 7.6 ,  0.5 ,  0.29, ...,  3.32,  0.62, 11.5 ],
       ...,
       [ 9.9 ,  0.32,  0.56, ...,  3.15,  0.73, 11.4 ],
       [ 8.7 ,  0.54,  0.26, ...,  3.27,  0.6 ,  9.3 ],
       [ 8.3 ,  0.54,  0.24, ...,  3.27,  0.61,  9.4 ]])

In [8]:
xgb_classification = xgb.XGBClassifier()
xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])

[17:16:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.46269
[1]	validation_0-mlogloss:1.30676
[2]	validation_0-mlogloss:1.19650
[3]	validation_0-mlogloss:1.11349
[4]	validation_0-mlogloss:1.05835
[5]	validation_0-mlogloss:1.01591
[6]	validation_0-mlogloss:0.98540
[7]	validation_0-mlogloss:0.96493
[8]	validation_0-mlogloss:0.95545
[9]	validation_0-mlogloss:0.93747
[10]	validation_0-mlogloss:0.92694
[11]	validation_0-mlogloss:0.92001
[12]	validation_0-mlogloss:0.90889
[13]	validation_0-mlogloss:0.90496
[14]	validation_0-mlogloss:0.90377
[15]	validation_0-mlogloss:0.90212
[16]	validation_0-mlogloss:0.89913
[17]	validation_0-mlogloss:0.90266
[18]	validation_0-mlogloss:0.90474
[19]	validation_0-mlogloss:0.90221

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[28]	validation_0-mlogloss:0.90815
[29]	validation_0-mlogloss:0.90782
[30]	validation_0-mlogloss:0.90552
[31]	validation_0-mlogloss:0.90688
[32]	validation_0-mlogloss:0.90947
[33]	validation_0-mlogloss:0.91186
[34]	validation_0-mlogloss:0.91212
[35]	validation_0-mlogloss:0.91447
[36]	validation_0-mlogloss:0.91669
[37]	validation_0-mlogloss:0.91837
[38]	validation_0-mlogloss:0.92217
[39]	validation_0-mlogloss:0.92255
[40]	validation_0-mlogloss:0.92585
[41]	validation_0-mlogloss:0.92807
[42]	validation_0-mlogloss:0.92823
[43]	validation_0-mlogloss:0.92915
[44]	validation_0-mlogloss:0.93027
[45]	validation_0-mlogloss:0.92970
[46]	validation_0-mlogloss:0.92851
[47]	validation_0-mlogloss:0.93168
[48]	validation_0-mlogloss:0.93258
[49]	validation_0-mlogloss:0.93349
[50]	validation_0-mlogloss:0.93967
[51]	validation_0-mlogloss:0.93856
[52]	validation_0-mlogloss:0.93900
[53]	validation_0-mlogloss:0.94059
[54]	validation_0-mlogloss:0.93925
[55]	validation_0-mlogloss:0.94190
[56]	validation_0-ml

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=32,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [11]:
def objective_classification(trial) :
    train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )
    
    param  = {
        'tree_method' :'gpu_hist',
        'verbosity' : 3,
        'objective' : "binary:logistics" , 
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)
        
    }
   
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x , test_y)

    
    
    return accuracy
        
        
    

In [12]:
xgb_classification_oputuna = optuna.create_study(direction  = 'minimize')
xgb_classification_oputuna.optimize(objective_classification , n_trials = 10 )

[I 2021-10-31 17:19:27,906] A new study created in memory with name: no-name-0bdf163f-871c-4c36-b0e9-416b8289c0dd


[17:19:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:19:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.31925
[1]	validation_0-mlogloss:1.25771
[2]	validation_0-mlogloss:1.22392
[3]	validation_0-mlogloss:1.20480
[4]	validation_0-mlogloss:1.19145
[5]	validation_0-mlogloss:1.18282
[6]	validation_0-mlogloss:1.1764

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[I 2021-10-31 17:19:28,077] Trial 0 finished with value: 0.459375 and parameters: {'booster': 'gblinear', 'lambda': 0.6579721472314581, 'alpha': 0.4665947267391802, 'subsample': 0.268012559738373, 'colsample_bytree': 0.12774574932033414}. Best is trial 0 with value: 0.459375.
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when const

[17:19:28] ======== Monitor (0): Learner ========
[17:19:28] Configure: 0.000485s, 1 calls @ 485us

[17:19:28] EvalOneIter: 0.006737s, 100 calls @ 6737us

[17:19:28] GetGradient: 0.006959s, 100 calls @ 6959us

[17:19:28] PredictRaw: 0.005327s, 100 calls @ 5327us

[17:19:28] UpdateOneIter: 0.044073s, 100 calls @ 44073us

[17:19:28] ======== Monitor (0): GBLinear ========
[17:19:28] DoBoost: 0.031057s, 100 calls @ 31057us

[17:19:28] PredictBatch: 0.007144s, 200 calls @ 7144us

[17:19:28] PredictBatchInternal: 0.006932s, 200 calls @ 6932us

[17:19:28] ======== Monitor (0): Learner ========
[17:19:28] Configure: 0.000135s, 1 calls @ 135us

[17:19:28] ======== Monitor (0): GBLinear ========
[17:19:28] PredictBatch: 3.6e-05s, 2 calls @ 36us

[17:19:28] PredictBatchInternal: 3.2e-05s, 2 calls @ 32us

[17:19:28] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/tree/updater_gpu_hist.cu:872: [GPU Hist]: Configure
[17:19:28] ======== Monitor (0): SketchContainer ========
[

[I 2021-10-31 17:20:46,684] Trial 1 finished with value: 0.55625 and parameters: {'booster': 'dart', 'lambda': 0.9008012602558423, 'alpha': 0.22386434968329705, 'subsample': 0.1505727447049417, 'colsample_bytree': 0.1313736065052783}. Best is trial 0 with value: 0.459375.


...

[17:24:34] Configure: 0.000507s, 1 calls @ 507us

[17:24:34] EvalOneIter: 0.011679s, 100 calls @ 11679us

[17:24:34] GetGradient: 0.008524s, 100 calls @ 8524us

[17:24:34] PredictRaw: 0.005517s, 100 calls @ 5517us

[17:24:34] UpdateOneIter: 0.068241s, 100 calls @ 68241us

[17:24:34] ======== Monitor (0): GBLinear ========
[17:24:34] DoBoost: 0.053426s, 100 calls @ 53426us

[17:24:34] PredictBatch: 0.010426s, 200 calls @ 10426us

[17:24:34] PredictBatchInternal: 0.010219s, 200 calls @ 10219us

[17:24:34] ======== Monitor (0): Learner ========
[17:24:34] Configure: 0.000149s, 1 calls @ 149us

[17:24:34] ======== Monitor (0): GBLinear ========
[17:24:34] PredictBatch: 2.3e-05s, 2 calls @ 23us

[17:24:34] PredictBatchInternal: 2.1e-05s, 2 calls @ 21us

[17:24:34] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/tree/updater_gpu_hist.cu:872: [GPU Hist]: Configure
[17:24:34] ======== Monitor (0): SketchContainer ========
[17:24:34] MakeCuts: 0.000815s, 1 calls @ 815us


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:24:34] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:904: drop 0 trees, weight = 1
[3]	validation_0-mlogloss:1.21261
[17:24:34] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:904: drop 0 trees, weight = 1
[4]	validation_0-mlogloss:1.15617
[17:24:34] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:904: drop 0 trees, weight = 1
[5]	validation_0-mlogloss:1.12314
[17:24:35] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:904: drop 0 trees, weight = 1
[6]	validation_0-mlogloss:1.08388
[17:24:35] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:904: drop 0 trees, weight = 1
[7]	validation_0-mlogloss:1.05692
[17:24:35] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:904: drop 0 trees, weight = 1
[8]	validation_0-mlogloss:1.04237
[17:24:35] INFO: C:/Users/Administrator/

[I 2021-10-31 17:25:50,260] Trial 9 finished with value: 0.625 and parameters: {'booster': 'dart', 'lambda': 0.05607209818856801, 'alpha': 0.3692618324552237, 'subsample': 0.4343461540241228, 'colsample_bytree': 0.25534318724334326}. Best is trial 0 with value: 0.459375.


[17:25:50] ======== Monitor (0): Learner ========
[17:25:50] Configure: 0.000755s, 1 calls @ 755us

[17:25:50] ======== Monitor (0): GBTree ========
[17:25:50] ======== Device 0 Memory Allocations:  ========
[17:25:50] Peak memory usage: 2MiB
[17:25:50] Number of allocations: 3149913
[17:25:50] ======== Monitor (0): updater_gpu_hist ========


In [13]:
xgb_classification_oputuna.best_trial

FrozenTrial(number=0, values=[0.459375], datetime_start=datetime.datetime(2021, 10, 31, 17, 19, 27, 908794), datetime_complete=datetime.datetime(2021, 10, 31, 17, 19, 28, 77791), params={'booster': 'gblinear', 'lambda': 0.6579721472314581, 'alpha': 0.4665947267391802, 'subsample': 0.268012559738373, 'colsample_bytree': 0.12774574932033414}, distributions={'booster': CategoricalDistribution(choices=('dart', 'gbtree', 'gblinear')), 'lambda': UniformDistribution(high=1.0, low=0.0001), 'alpha': UniformDistribution(high=1.0, low=0.0001), 'subsample': UniformDistribution(high=0.5, low=0.1), 'colsample_bytree': UniformDistribution(high=0.5, low=0.1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)

In [15]:
param = xgb_classification_oputuna.best_trial.params

In [16]:
xgb_final_clas = xgb.XGBClassifier(**param)
xgb_final_clas.fit(train_x , train_y)

[17:30:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:30:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:30:01] ======== Monitor (0): Learner ========
[17:30:01] Configure: 0.000488s, 1 calls @ 488us

[17:30:01] EvalOneIter: 0.000476s, 100 calls @ 476us

[17:30:01] GetGradient: 0.006313s, 100 calls @ 6313us

[17:30:01] PredictRaw: 0.002537s, 100 call

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(alpha=0.4665947267391802, base_score=0.5, booster='gblinear',
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.12774574932033414, enable_categorical=False,
              gamma=None, gpu_id=-1, importance_type=None,
              interaction_constraints=None, lambda=0.6579721472314581,
              learning_rate=0.5, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=32, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, random_state=0,
              reg_alpha=0, reg_lambda=0, scale_pos_weight=None,
              subsample=0.268012559738373, tree_method=None,
              validate_parameters=1, ...)

In [17]:
xgb_final_clas.score(test_x , test_y)

0.4